In [1]:
#######################################################################################################
# Summary
# 1. Tensorflow Multi-GPU example using Estimator & Dataset high-APIs
# 2. On-the-fly data-augmentation (random crop, random flip)
# ToDo:
# 2. Add validation-data to estimator.train()
# 3. Investigate tfrecord speed improvement (to match MXNet)
# References:
# https://www.tensorflow.org/performance/performance_guide
# 1. https://jhui.github.io/2017/03/07/TensorFlow-Perforamnce-and-advance-topics/
# 2. https://www.tensorflow.org/versions/master/performance/datasets_performance
# 3. https://github.com/pudae/tensorflow-densenet
# 4. https://stackoverflow.com/a/48096625/6772173
# 5. https://stackoverflow.com/questions/47867748/transfer-learning-with-tf-estimator-estimator-framework
# 6. https://github.com/BobLiu20/Classification_Nets/blob/master/tensorflow/common/average_gradients.py
# 7. https://github.com/BobLiu20/Classification_Nets/blob/master/tensorflow/training/train_estimator.py
#######################################################################################################

In [2]:
MULTI_GPU = True  # TOGGLE THIS

In [3]:
%%bash 
# Download model check-point and module from below repo:
#wget -N https://github.com/pudae/tensorflow-densenet/raw/master/nets/densenet.py
#wget -N https://ikpublictutorial.blob.core.windows.net/deeplearningframeworks/tf-densenet121.tar.gz
#tar xzvf tf-densenet121.tar.gz

In [4]:
import os
import sys
import time
import multiprocessing
import numpy as np
import pandas as pd
from PIL import Image
import random
import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow.python.framework.ops import convert_to_tensor
from tensorflow.contrib.data import Iterator
from common.utils import download_data_chextxray, get_imgloc_labels, get_train_valid_test_split
from common.utils import compute_roc_auc, get_cuda_version, get_cudnn_version, get_gpu_name
from common.params_dense import *
slim = tf.contrib.slim
import densenet  # Download from https://github.com/pudae/tensorflow-densenet

In [5]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.2
Tensorflow:  1.6.0
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 9.0.176
CuDNN Version  7.0.5


In [6]:
CPU_COUNT = multiprocessing.cpu_count()
GPU_COUNT = len(get_gpu_name())
print("CPUs: ", CPU_COUNT)
print("GPUs: ", GPU_COUNT)

CPUs:  24
GPUs:  4


In [7]:
# Model-params
IMAGENET_RGB_MEAN_CAFFE = np.array([123.68, 116.78, 103.94], dtype=np.float32)
IMAGENET_SCALE_FACTOR_CAFFE = 0.017
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)
CHKPOINT = 'tf-densenet121.ckpt'  # Downloaded tensorflow-checkpoint

chestxray/images chestxray/Data_Entry_2017.csv


In [8]:
# Manually scale to multi-gpu
if MULTI_GPU:
    LR *= GPU_COUNT 
    BATCHSIZE *= GPU_COUNT

In [9]:
%%time
# Download data
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 659 ms, sys: 234 ms, total: 893 ms
Wall time: 892 ms


In [10]:
#####################################################################################################
## Data Loading

In [11]:
class XrayData():
    
    def __init__(self, img_dir, lbl_file, patient_ids, mode='inference', 
                 width=WIDTH, height=HEIGHT, batch_size=BATCHSIZE, 
                 imagenet_mean=IMAGENET_RGB_MEAN_CAFFE, imagenet_scaling = IMAGENET_SCALE_FACTOR_CAFFE,
                 buffer=10):

        self.img_locs, self.labels = get_imgloc_labels(img_dir, lbl_file, patient_ids)
        self.data_size = len(self.labels)
        self.imagenet_mean = imagenet_mean
        self.imagenet_scaling = imagenet_scaling
        self.width = width
        self.height = height
        data = tf.data.Dataset.from_tensor_slices((self.img_locs, self.labels))
        
        # Processing
        # Output as channels-last and TF model will reshape in densenet.py
        # inputs = tf.transpose(inputs, [0, 3, 1, 2])
        if mode == 'training':
            data = data.shuffle(self.data_size).repeat().apply(
                tf.contrib.data.map_and_batch(self._parse_function_train, batch_size)).prefetch(buffer)
        elif mode == 'inference':
            data = data.apply(
                tf.contrib.data.map_and_batch(self._parse_function_inference, batch_size)).prefetch(buffer)
        
        self.data = data        
        print("Loaded {} labels and {} images".format(len(self.labels), len(self.img_locs)))
        
        
    def _parse_function_train(self, filename, label):
        img_rgb, label = self._preprocess_image_labels(filename, label)
        # Random crop (from 264x264)
        img_rgb = tf.image.resize_images(img_rgb, [self.height+40, self.width+40])
        img_rgb = tf.random_crop(img_rgb, [self.height, self.width, 3])
        # Random flip
        img_rgb = tf.image.random_flip_left_right(img_rgb)
        # Channels-first
        img_rgb = tf.transpose(img_rgb, [2, 0, 1])
        return img_rgb, label
        
        
    def _parse_function_inference(self, filename, label):
        img_rgb, label = self._preprocess_image_labels(filename, label)
        # Resize to final dimensions
        img_rgb = tf.image.resize_images(img_rgb, [self.height, self.width])
        # Channels-first
        img_rgb = tf.transpose(img_rgb, [2, 0, 1])
        return img_rgb, label 
       
    
    def _preprocess_image_labels(self, filename, label):
        # load and preprocess the image
        img_decoded = tf.to_float(tf.image.decode_png(tf.read_file(filename), channels=3))
        img_centered = tf.subtract(img_decoded, self.imagenet_mean)
        img_rgb = img_centered * self.imagenet_scaling
        return img_rgb, tf.cast(label, dtype=tf.float32)

In [12]:
train_set, valid_set, test_set = get_train_valid_test_split(TOT_PATIENT_NUMBER)

train:21563 valid:3080 test:6162


In [13]:
with tf.device('/cpu:0'):
    # Create dataset for iterator
    train_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=train_set,  
                             mode='training')
    valid_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=valid_set)
    test_dataset  = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=test_set)

Loaded 87306 labels and 87306 images
Loaded 7616 labels and 7616 images
Loaded 17198 labels and 17198 images


In [14]:
#####################################################################################################
## Helper Functions

In [15]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        grads = []
        for g, _ in grad_and_vars:
            expanded_g = tf.expand_dims(g, 0)
            grads.append(expanded_g)
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [16]:
def get_symbol(model_name, in_tensor, is_training, chkpoint, out_features):
    if model_name == 'densenet121':
         # Import symbol
        dense_args = densenet.densenet_arg_scope(data_format="NCHW")
        # Maybe also needs fused batch-norm?
        print(dense_args)
        with slim.arg_scope(dense_args):
            base_model, _ = densenet.densenet121(in_tensor,
                                                 num_classes=out_features,
                                                 is_training=is_training)
            # Need to reshape from (?, 1, 1, 14) to (?, 14)
            sym = tf.reshape(base_model, shape=[-1, out_features])
        # Collect variables to restore from checkpoint
        variables_to_restore = slim.get_variables_to_restore(exclude=['densenet121/logits', 'predictions'])
    else:
        raise ValueError("Unknown model-name")
    return sym, variables_to_restore

In [17]:
def model_fn(features, labels, mode, params):
    # Create symbol
    # is_training=True? | https://github.com/tensorflow/models/issues/3556
    sym, variables_to_restore = get_symbol(
        model_name=params["model_name"],
        in_tensor=features, 
        #is_training=(mode == tf.estimator.ModeKeys.TRAIN),
        is_training=True,
        chkpoint=params["checkpoint"],
        out_features=params["n_classes"])
    # ModeKeys.PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=tf.sigmoid(sym))
    # Optimizer & Loss
    optimizer = tf.train.AdamOptimizer(params['lr'], beta1=0.9, beta2=0.999)
    loss_fn = tf.losses.sigmoid_cross_entropy(labels, sym)
    loss = tf.reduce_mean(loss_fn)
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.train.init_from_checkpoint(params['checkpoint'], 
                              {v.name.split(':')[0]: v for v in variables_to_restore})
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=optimizer.minimize(loss, tf.train.get_or_create_global_step()))

In [18]:
def model_fn_multigpu(features, labels, mode, params):
    if mode == tf.estimator.ModeKeys.PREDICT:
        # Create symbol
        sym, _ = get_symbol(
            model_name=params["model_name"],
            in_tensor=features, 
            is_training=True,
            chkpoint=params["checkpoint"],
            out_features=params["n_classes"])
        
        # Predictions
        predictions = tf.sigmoid(sym)   
        # ModeKeys.PREDICT
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    elif mode == tf.estimator.ModeKeys.TRAIN:
        # Make sure splits sum to batch-size
        split_size = params["batchsize"] // len(params["gpus"])
        splits = [split_size, ] * (len(params["gpus"]) - 1)
        splits.append(params["batchsize"] - split_size * (
            len(params["gpus"]) - 1))
        # Split the features and labels
        features_split = tf.split(features, splits, axis=0)
        labels_split = tf.split(labels, splits, axis=0)
        tower_grads = []
        # Training operation
        global_step = tf.train.get_global_step()
        optimizer = tf.train.AdamOptimizer(LR, beta1=0.9, beta2=0.999)
        # Load model on multiple GPUs
        restore_list = []
        with tf.variable_scope(tf.get_variable_scope()):
            for i in range(len(params['gpus'])):
                with tf.device('/gpu:%d' % i):
                    with tf.name_scope('%s_%d' % ("classification", i)) as scope:
                        # Symbol
                        sym, variables_to_restore = get_symbol(
                            model_name=params["model_name"],
                            in_tensor=features_split[i], 
                            is_training=True,
                            chkpoint=params["checkpoint"],
                            out_features=params["n_classes"])
                        # Since reuse only need [0]
                        restore_list.append(variables_to_restore)
                        # Loss
                        tf.losses.sigmoid_cross_entropy(labels_split[i], sym)
                        # Training-ops
                        update_ops = tf.get_collection(
                            tf.GraphKeys.UPDATE_OPS, scope)
                        updates_op = tf.group(*update_ops)
                        with tf.control_dependencies([updates_op]):
                            losses = tf.get_collection(tf.GraphKeys.LOSSES, scope)
                            print(losses)
                            total_loss = tf.add_n(losses, name='total_loss')
                        # reuse var
                        tf.get_variable_scope().reuse_variables()
                        # grad compute
                        grads = optimizer.compute_gradients(total_loss)
                        tower_grads.append(grads)

        # We must calculate the mean of each gradient
        grads = average_gradients(tower_grads)
        # Apply the gradients to adjust the shared variables.
        apply_gradient_op = optimizer.apply_gradients(grads, global_step=global_step)
        # Group all updates to into a single train op.
        train_op = tf.group(apply_gradient_op)
        # Initialise from checkpoint
        tf.train.init_from_checkpoint(params['checkpoint'], 
                              {v.name.split(':')[0]: v for v in restore_list[0]})
        
        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss=total_loss,
            train_op=train_op)

In [19]:
def train_input_fn():
    return train_dataset.data.make_one_shot_iterator().get_next()
def valid_input_fn():
    return valid_dataset.data.make_one_shot_iterator().get_next()
def test_input_fn():
    return test_dataset.data.make_one_shot_iterator().get_next()

In [20]:
%%time
# Create Estimator
if MULTI_GPU:
    nn = tf.estimator.Estimator(model_fn=model_fn_multigpu,
                                params={"lr":LR, 
                                        "checkpoint":CHKPOINT,
                                        "n_classes":CLASSES,
                                        "batchsize":BATCHSIZE,
                                        "gpus":list(range(GPU_COUNT)),
                                        "model_name":"densenet121"})
else:
    nn = tf.estimator.Estimator(model_fn=model_fn,
                                params={"lr":LR, 
                                        "checkpoint":CHKPOINT,
                                        "n_classes":CLASSES,
                                        "model_name":"densenet121"})    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_summary_steps': 100, '_session_config': None, '_tf_random_seed': None, '_task_id': 0, '_evaluation_master': '', '_service': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8590f6b7f0>, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_master': '', '_model_dir': '/tmp/tmpizx8jhl7', '_keep_checkpoint_max': 5, '_global_id_in_cluster': 0}
CPU times: user 4.42 ms, sys: 110 µs, total: 4.53 ms
Wall time: 4.14 ms


In [21]:
%%time
# 1 GPU - Main training loop: 62min 8s
# 2 GPU - Main training loop: 44min 13s
# 4 GPU - Main training loop: 31min 4s
# What's a good way of adding validation data here?
nn.train(train_input_fn, 
         max_steps=EPOCHS*(train_dataset.data_size//BATCHSIZE))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable global_step:0 from checkpoint tf-densenet121.ckpt with global_step
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpizx8jhl7/model.ckpt.
INFO:tensorflow:step = 1, loss = 0.8039173
INFO:tensorflow:global_step/sec: 0.90518
INFO:tensorflow:step = 101, loss = 0.16633654 (110.478 sec)
INFO:tensorflow:global_step/sec: 0.977861
INFO:tensorflow:step = 201, loss = 0.16473089 (102.264 sec)
INFO:tensorflow:global_step/sec: 0.97399
INFO:tensorflow:step = 301, loss = 0.16107056 (102.673 sec)
INFO:tensorflow:global_step/sec: 0.987141
INFO:tensorflow:step = 401, loss = 0.16161034 (101.300 sec)
INFO:tensorflow:global_step/sec: 0.982739
INFO:tensorflow:step = 501, loss = 0.114540115 (101.757 sec)
INFO:tensorflow:Saving checkpoint

In [22]:
%%time
# Main prediction loop: 1min
# Test AUC: 0.8150
predictions = list(nn.predict(test_input_fn))
y_truth = test_dataset.labels
y_guess = np.array(predictions)
print("Test AUC: {0:.4f}".format(compute_roc_auc(y_truth, y_guess, CLASSES))) 

INFO:tensorflow:Calling model_fn.
{'<function batch_norm at 0x7f85a00b0d90>': {'decay': 0.99, 'fused': True, 'scale': True, 'epsilon': 1.1e-05, 'data_format': 'NCHW'}, '<function convolution at 0x7f85a00a30d0>': {'activation_fn': None, 'weights_regularizer': <function l2_regularizer.<locals>.l2 at 0x7f85927be488>, 'biases_initializer': None, 'data_format': 'NCHW'}, '<function _global_avg_pool2d at 0x7f85927b1510>': {'data_format': 'NCHW'}, '<function _conv_block at 0x7f85927b1730>': {'data_format': 'NCHW'}, '<function max_pool2d at 0x7f859fded2f0>': {'data_format': 'NCHW'}, '<function avg_pool2d at 0x7f85a0089bf8>': {'data_format': 'NCHW'}}
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpizx8jhl7/model.ckpt-1705
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Full AUC [0.8102469725158122, 0.8603340664566368, 0.8016955441462638, 0.8885381081992122, 0.8790134552257802, 0.91141

In [ ]:
#####################################################################################################
## Synthetic Data (Pure Training)

In [23]:
# Test on fake-data -> no IO lag
batch_in_epoch = train_dataset.data_size//BATCHSIZE
tot_num = batch_in_epoch * BATCHSIZE
fake_X = np.random.rand(tot_num, 3, 224, 224).astype(np.float32)
fake_y = np.random.rand(tot_num, CLASSES).astype(np.float32) 

In [37]:
%%time
# Create Estimator
if MULTI_GPU:
    nn = tf.estimator.Estimator(model_fn=model_fn_multigpu,
                                params={"lr":LR, 
                                        "checkpoint":CHKPOINT,
                                        "n_classes":CLASSES,
                                        "batchsize":BATCHSIZE,
                                        "gpus":list(range(GPU_COUNT)),
                                        "model_name":"densenet121"})
else:
    nn = tf.estimator.Estimator(model_fn=model_fn,
                                params={"lr":LR, 
                                        "checkpoint":CHKPOINT,
                                        "n_classes":CLASSES,
                                        "model_name":"densenet121"})    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_summary_steps': 100, '_session_config': None, '_tf_random_seed': None, '_task_id': 0, '_evaluation_master': '', '_service': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f858125bf60>, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_master': '', '_model_dir': '/tmp/tmpp248ho0x', '_keep_checkpoint_max': 5, '_global_id_in_cluster': 0}
CPU times: user 628 µs, sys: 4.5 ms, total: 5.13 ms
Wall time: 4.57 ms


In [38]:
%%time
# 4 GPU - Main training loop: 31min 4s
# 4 GPU - Synthetic data: 17min 10s
nn.train(tf.estimator.inputs.numpy_input_fn(
    fake_X,
    fake_y,
    shuffle=False,
    num_epochs=EPOCHS,
    batch_size=BATCHSIZE))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable global_step:0 from checkpoint tf-densenet121.ckpt with global_step
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpp248ho0x/model.ckpt.
INFO:tensorflow:step = 1, loss = 0.80244374
INFO:tensorflow:global_step/sec: 1.6309
INFO:tensorflow:step = 101, loss = 0.693112 (61.319 sec)
INFO:tensorflow:global_step/sec: 1.86052
INFO:tensorflow:step = 201, loss = 0.693772 (53.749 sec)
INFO:tensorflow:global_step/sec: 1.85498
INFO:tensorflow:step = 301, loss = 0.6959225 (53.908 sec)
INFO:tensorflow:global_step/sec: 1.85498
INFO:tensorflow:step = 401, loss = 0.69358575 (53.911 sec)
INFO:tensorflow:global_step/sec: 1.82027
INFO:tensorflow:step = 501, loss = 0.6940733 (54.935 sec)
INFO:tensorflow:global_step/sec: 1.83615
INFO:te